In [1]:
#GOAL: Extract from kickstarter database a sample of tables, 
# based on projects in Santa Monica
import sqlite3
import pandas as pd
import mysql.connector

In [2]:
#PROJECTS
#Convert a projects query into a projects dataframe

def sqlite_sample(sql, columns):
    '''Selects from sqlite database, and creates dataframe as result
    sql - the SQL SELECT statement; a tuple containing the query as a string.
    columns - list of columns to pull'''
    conn = sqlite3.connect('/Users/markmann/Desktop/INF551_Homework/Project/dataverse_files/kickstarter.db')
    cur = conn.cursor()

    cur.execute(sql)
    response = cur.fetchall()
    projects_df = pd.DataFrame(response, columns=columns)
    conn.close()

#     #CREATE PROJECTS TABLE
#     conn = sqlite3.connect('/Users/markmann/Desktop/INF551_Homework/Project/kickstarter_samp.db')
#     cur = conn.cursor()
#     cur.execute('DROP TABLE IF EXISTS project')
#     cur.execute('CREATE TABLE project (project_id INTEGER, name TEXT, state TEXT, country TEXT, creator_id INTEGER, location_id INTEGER, category_id INTEGER, goal INTEGER, pledged REAL, backers_count INTEGER, blurb TEXT)')

#     #UPDATE PROJECT TABLE
#     for i in range(projects_df.shape[0]):
#         cur.execute('INSERT INTO project (project_id, name, state, country, creator_id, location_id, category_id, goal, pledged, backers_count, blurb) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)',
#                    (int(projects_df['project_id'][i]), projects_df['name'][i], projects_df['state'][i], projects_df['country'][i], int(projects_df['creator_id'][i]), int(projects_df['location_id'][i]), int(projects_df['category_id'][i]), projects_df['goal'][i], projects_df['pledged'][i], int(projects_df['backers_count'][i]), projects_df['blurb'][i]))

    return projects_df

In [3]:
def add_to_mysql(projects_df):
    '''Takes dataframe, and adds it to mysql
    Assumes for the PROJECT table only, at this point. 
    '''
    #CONNECT TO MYSQL
    mydb = mysql.connector.connect(host="localhost",
                                   user="root",
                                   password='testpassword123',
                                   database='kickstarter_samp',
                                   auth_plugin='mysql_native_password'
                                  )
    mycursor = mydb.cursor()

    #CREATE MYSQL PROJECTS TABLE
    mycursor.execute("DROP TABLE IF EXISTS project")
    mycursor.execute("CREATE TABLE project (project_id INT(10), name VARCHAR(200), state VARCHAR(2), country VARCHAR(2), creator_id INT(10), location_id INT(7), category_id INT(3), goal INT(6), pledged FLOAT(10,2), backers_count INT(4), blurb VARCHAR(200))")

    #INSERT INTO MYSQL PROJECT TABLE
    sql = 'INSERT INTO project (project_id, name, state, country, creator_id, location_id, category_id, goal, pledged, backers_count, blurb) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'
    for i in range(projects_df.shape[0]):
        val = (str(projects_df['project_id'][i]), str(projects_df['name'][i]), str(projects_df['state'][i]), str(projects_df['country'][i]), str(projects_df['creator_id'][i]), str(projects_df['location_id'][i]), str(projects_df['category_id'][i]), str(projects_df['goal'][i]), str(projects_df['pledged'][i]), str(projects_df['backers_count'][i]), str(projects_df['blurb'][i]))
        mycursor.execute(sql, val)
    mydb.commit()

In [4]:
def project_driver():
    sql_project = ('SELECT project.id, project.name, location.state, project.country, project.creator_id, project.location_id, project.category_id, project.goal, project.pledged, project.backers_count, project.blurb '
                   'FROM project '
                   'JOIN location on project.location_id = location.id '
                   'WHERE location.country = "US" AND location.state = "CA" AND location.id = "2488892"'
                   'ORDER BY project.last_modification ')
    columns = ['project_id', 'name', 'state', 'country', 'creator_id', 'location_id', 'category_id', 'goal', 'pledged', 'backers_count', 'blurb']
    projects_df = sqlite_sample(sql_project, columns)
    add_to_mysql(projects_df)

In [5]:
#CHECK IF THE DATA IS THERE
mydb = mysql.connector.connect(host="localhost",
                                   user="root",
                                   password='testpassword123',
                                   database='kickstarter_samp',
                                   auth_plugin='mysql_native_password'
                                  )
mycursor = mydb.cursor()

mycursor.execute("SELECT * FROM project")
myresult = mycursor.fetchall()
print(myresult)

[(47344828, "Charlie Gee's Music Video", 'CA', 'US', 855660763, 2488892, 39, 420, 0.0, 0, 'Help me fund my next official music video directed by Steven Moga :] \nI will cherish your pledges!'), (47464951, 'Bring the Bongos to the Rathole', 'CA', 'US', 928839330, 2488892, 33, 2525, 320.0, 4, "His movies went belly-up on 3 continents. Now it's getting personal. This is a one-man's journey to make a $200M Hollywood blockbuster."), (53955113, 'FORMA - the Cheese Restaurant you were waiting for!', 'CA', 'US', 159093607, 2488892, 312, 30000, 0.0, 0, 'An extraordinary culinary experience that will give you the ultimate umami taste. Be part of it!'), (55277281, 'Animation Short Film: promoting nature and wildlife', 'CA', 'US', 580901810, 2488892, 290, 845, 71.0, 4, 'Help me to create a musically animated short film showcasing the beauty of nature at its heart, and promoting wildlife conservation.'), (63759178, 'Patricia', 'CA', 'US', 8066021, 2488892, 32, 4500, 5145.0, 67, "Patricia is trying 

In [ ]:
#CREATOR
#Run creator sql query and store in dataframe
def creator_sample():
    conn = sqlite3.connect('/Users/markmann/Desktop/INF551_Homework/Project/dataverse_files/kickstarter.db')
    cur = conn.cursor()
    sql = ('SELECT DISTINCT creator.id, creator.name, creator.slug '
           'FROM creator '
           'JOIN (SELECT project.id, project.creator_id '
               'FROM project '
               'JOIN location on project.location_id = location.id '
               'WHERE location.country = "US" AND location.state = "CA" AND location.id = "2488892") ca_projects on creator.id = ca_projects.creator_id '
            )
    cur.execute(sql)
    response = cur.fetchall()
    creator_df = pd.DataFrame(response, columns=['creator_id', 'name', 'slug'])
    #creator_df.head()

    #CREATE CREATOR TABLE
    conn = sqlite3.connect('/Users/markmann/Desktop/INF551_Homework/Project/kickstarter_samp.db')
    cur = conn.cursor()
    cur.execute('DROP TABLE IF EXISTS creator')
    cur.execute('CREATE TABLE creator (creator_id INTEGER, name TEXT, slug TEXT)')

    #UPDATE CREATOR TABLE
    for i in range(creator_df.shape[0]):
        cur.execute('INSERT INTO creator (creator_id, name, slug) VALUES (?, ?, ?)',
                   (int(creator_df['creator_id'][i]), creator_df['name'][i], creator_df['slug'][i]))


    conn.commit()
    conn.close()
    
    return creator_df

In [ ]:
#COMMENTS
#Run creator sql query and store in dataframe
def creator_sample():
    conn = sqlite3.connect('/Users/markmann/Desktop/INF551_Homework/Project/dataverse_files/kickstarter.db')
    cur = conn.cursor()
    sql = ('SELECT comments.id, comments.projectid, comments.user_name, comments.body '
           'FROM comments '
           'JOIN (SELECT project.id project_id '
               'FROM project '
               'JOIN location on project.location_id = location.id '
               'WHERE location.country = "US" AND location.state = "CA" AND location.id = "2488892") ca_projects on comments.projectid = ca_projects.project_id '
          )
    cur.execute(sql)
    response = cur.fetchall()
    comments_df = pd.DataFrame(response, columns=['comment_id', 'projectid', 'user_name', 'body'])

    cur.execute('DROP TABLE IF EXISTS comments')
    cur.execute('CREATE TABLE comments (location_id INTEGER, name TEXT, slug TEXT)')

    #CREATE COMMENTS TABLE
    conn = sqlite3.connect('/Users/markmann/Desktop/INF551_Homework/Project/kickstarter_samp.db')
    cur = conn.cursor()
    cur.execute('DROP TABLE IF EXISTS comments')
    cur.execute('CREATE TABLE comments (comment_id INTEGER, projectid INTEGER, user_name TEXT, body TEXT)')

    #UPDATE CREATOR TABLE
    for i in range(comments_df.shape[0]):
        cur.execute('INSERT INTO comments (comment_id, projectid, user_name, body) VALUES (?, ?, ?, ?)',
                   (int(comments_df['comment_id'][i]), comments_df['projectid'][i], comments_df['user_name'][i], comments_df['body'][i]))


    conn.commit()
    conn.close()
    
    return comments_df

In [ ]:
#LOCATION

In [ ]:
#Get the projects_df
project_df = projects_sample()
project_df.shape

In [ ]:
#GOAL: Create a MySQL DB and PROJECTS table from SQLite

#Make a new connection to the MySQL server
mydb = mysql.connector.connect(host="localhost",
                               user="root",
                               password='testpassword123',
                               database='kickstarter_samp',
                               auth_plugin='mysql_native_password'
                              )
mycursor = mydb.cursor()

#Check dbs that exist
# mycursor.execute("SHOW DATABASES")
# for x in mycursor:
#     print(x)



#Create database
#mycursor.execute("CREATE DATABASE kickstarter_samp")

#Delete kickstarter db

#Create table
mycursor.execute("CREATE TABLE project (project_id INT(10), name VARCHAR(200), state VARCHAR(2), country VARCHAR(2), creator_id INT(10), location_id INT(7), category_id INT(3), goal INT(6), pledged FLOAT(8,2), backers_count INT(4), blurb VARCHAR(200))")

#Delete table

#Insert into table
for i in range(projects_df.shape[0]):

In [ ]:
#Insert from dataframe to project table
sql = "INSERT INTO projects (name, address) VALUES (%s, %s)"